In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from hidmed import *
import pickle
import matplotlib.pyplot as plt

In [3]:
def tune_parameters(xdim, zdim, wdim, mdim, udim, setup, n, folds=1, seed=0):
    """Tune the parameters of the estimators for a given setup."""
    assert setup in ["a", "b", "c"], "Invalid setup. Must be 'a', 'b', or 'c'."
    datagen = LinearHidMedDGP(xdim, zdim, wdim, mdim, udim, setup=setup, seed=seed)
    dataset = datagen.sample_dataset(n, seed=seed + 1)
    estimator = ProximalMultiplyRobust(
        generalized_model="c",
        folds=folds,
        num_runs=200,
        n_jobs=1,
    )
    estimator.fit(dataset)
    
    ProximalMultiplyRobust(
        generalized_model="c",
        folds=folds,
        num_runs=200,
        n_jobs=1,
        treatment=estimator.params["treatment"],
        h=estimator.params["h"],
        q=estimator.params["q"],
        eta=estimator.params["eta"],
    ).fit(dataset)
    print("^PMR")

    ProximalOutcomeRegression(
        generalized_model=(setup == "b" or setup == "c"),
        folds=folds,
        num_runs=200,
        n_jobs=1,
        treatment=estimator.params["treatment"],
        h=estimator.params["h"],
        eta=estimator.params["eta"],
    ).fit(dataset)
    print("^POR")

    ProximalInverseProbWeighting(
        generalized_model=(setup == "b" or setup == "c"),
        folds=folds,
        num_runs=200,
        n_jobs=1,
        q=estimator.params["q"],
    ).fit(dataset)
    print("^PIPW")

    print("true psi:", datagen.true_psi())
    return estimator.params


# tuned_parameters = {}
# for n in np.arange(300, 7000, 900):
#     # 1d-case
#     print("1d-case, n =", n)
#     tuned_parameters[1, n] = tune_parameters(1, 1, 1, 1, 1, "c", n, folds=2)
#     # 5d-case
#     print("5d-case, n =", n)
#     tuned_parameters[5, n] = tune_parameters(5, 2, 2, 2, 1, "c", n, folds=2)
#     print("\n")

# pickle.dump(tuned_parameters, open("assets/tuned_parameters.pkl", "wb"))

In [4]:
tune_parameters(1, 1, 1, 1, 1, "a", 3000, folds=2)

==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 0.0001, 'lambda2': 5703199.7181949, 'gamma': 0.0297475} score:  -1.1e-06


/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/hom

/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/hom

Treatment prob params: {'C': 1000000.0, 'degree': 1} log_loss:  0.647
Bridge q params: {'lambda1': 0.0005153, 'lambda2': 10000000.0, 'gamma': 0.0006049} score:  -3e-07
eta params: {'alpha': 0.083, 'gamma': 4.466} r2:  0.983
==== Estimate 1: 0.15060253735265774
==== Cross-fitting fold 2 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 0.0001, 'lambda2': 5703199.7181949, 'gamma': 0.0297475} score:  nan
Treatment prob params: {'C': 1000000.0, 'degree': 1} log_loss:  nan
Bridge q params: {'lambda1': 0.0005153, 'lambda2': 10000000.0, 'gamma': 0.0006049} score:  nan
eta params: {'alpha': 0.083, 'gamma': 4.466} r2:  nan
==== Estimate 2: 0.19233300810680715
==== Estimate: 0.17146777272973246
==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 0.0001, 'lambda2': 5703199.7181949, 'gamma': 0.0297475} score:  nan
Treatment prob params: {'C': 1000000.0, 'degree': 1} log_loss:  nan
Bridge q params: {'lambda1': 0.0005153, 'lambda2': 10000000.0, 'gamma': 0.0006049} score:  nan

{'treatment': {'C': 999999.9999999995, 'degree': 1},
 'h': {'lambda1': 9.999999999999991e-05,
  'lambda2': 5703199.7181949,
  'gamma': 0.02974752241107422},
 'q': {'lambda1': 0.0005152558350987109,
  'lambda2': 10000000.000000006,
  'gamma': 0.0006049134315092761},
 'eta': {'alpha': 0.083380797668556, 'gamma': 4.4660821242259585}}

In [5]:
tune_parameters(1, 1, 1, 1, 1, "b", 3000, folds=2)

==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 28365.436189, 'lambda2': 10000000.0, 'gamma': 0.0001649} score:  -3.3e-06
Treatment prob params: {'C': 1000000.0, 'degree': 2} log_loss:  0.652
Bridge q params: {'lambda1': 0.0001, 'lambda2': 10000000.0, 'gamma': 100.0} score:  -3e-07
eta params: {'alpha': 0.0, 'gamma': 0.083} r2:  0.974
==== Estimate 1: -2.2812447048294247e-07
==== Cross-fitting fold 2 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 28365.436189, 'lambda2': 10000000.0, 'gamma': 0.0001649} score:  nan
Treatment prob params: {'C': 1000000.0, 'degree': 2} log_loss:  nan
Bridge q params: {'lambda1': 0.0001, 'lambda2': 10000000.0, 'gamma': 100.0} score:  nan
eta params: {'alpha': 0.0, 'gamma': 0.083} r2:  nan
==== Estimate 2: 6.9100665167722105e-06
==== Estimate: 3.340971023144634e-06
==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 28365.436189, 'lambda2': 10000000.0, 'gamma': 0.0001649} score:  nan
Treatment prob para

{'treatment': {'C': 999999.9999999995, 'degree': 2},
 'h': {'lambda1': 28365.436189038406,
  'lambda2': 10000000.000000006,
  'gamma': 0.00016487284992298685},
 'q': {'lambda1': 9.999999999999991e-05,
  'lambda2': 10000000.000000006,
  'gamma': 100.00000000000004},
 'eta': {'alpha': 3.936472099787732e-06, 'gamma': 0.0828211914607539}}

In [6]:
tune_parameters(1, 1, 1, 1, 1, "c", 3000, folds=2)

==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 673846.6261991, 'lambda2': 10000000.0, 'gamma': 47.024831} score:  -1.7e-06


/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Treatment prob params: {'C': 1000000.0, 'degree': 2} log_loss:  0.652
Bridge q params: {'lambda1': 0.0001, 'lambda2': 9687299.3223714, 'gamma': 0.0001083} score:  -1e-07
eta params: {'alpha': 1.514, 'gamma': 21.544} r2:  0.059
==== Estimate 1: 0.40288442664235774
==== Cross-fitting fold 2 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 673846.6261991, 'lambda2': 10000000.0, 'gamma': 47.024831} score:  nan
Treatment prob params: {'C': 1000000.0, 'degree': 2} log_loss:  nan
Bridge q params: {'lambda1': 0.0001, 'lambda2': 9687299.3223714, 'gamma': 0.0001083} score:  nan
eta params: {'alpha': 1.514, 'gamma': 21.544} r2:  nan
==== Estimate 2: 0.43817512221273885
==== Estimate: 0.4205297744275483
==== Cross-fitting fold 1 (1000/2000 fit/eval)
Bridge h params: {'lambda1': 673846.6261991, 'lambda2': 10000000.0, 'gamma': 47.024831} score:  nan
Treatment prob params: {'C': 1000000.0, 'degree': 2} log_loss:  nan
Bridge q params: {'lambda1': 0.0001, 'lambda2': 9687299.3223714, 'gamma': 0.0001083

{'treatment': {'C': 999999.9999999995, 'degree': 2},
 'h': {'lambda1': 673846.6261990658,
  'lambda2': 10000000.000000006,
  'gamma': 47.02483098427903},
 'q': {'lambda1': 9.999999999999991e-05,
  'lambda2': 9687299.32237145,
  'gamma': 0.00010826324554063362},
 'eta': {'alpha': 1.5136134960784886, 'gamma': 21.543525653616125}}

In [ ]:
tune_parameters(5, 2, 2, 2, 1, "a", 6000, folds=2)

==== Cross-fitting fold 1 (2000/4000 fit/eval)


In [ ]:
tune_parameters(5, 2, 2, 2, 1, "b", 6000, folds=2)

In [ ]:
tune_parameters(5, 2, 2, 2, 1, "c", 6000, folds=2)

In [ ]:
tune_parameters(5, 2, 2, 2, 1, "a", 3000, folds=2)

In [ ]:
tune_parameters(5, 2, 2, 2, 1, "b", 3000, folds=2)

In [ ]:
tune_parameters(5, 2, 2, 2, 1, "c", 3000, folds=2)